In [4]:
%run ../utils/init_env.py

In [10]:
import requests
import config
import json
import csv
import random
import re
import os
from datetime import datetime
from typing import List, Dict
from concurrent.futures import ThreadPoolExecutor

# Configuration
API_URL = config.get_model_config()['api_url']
print(API_URL)

http://Adrians-Mac-mini.local:12345/v1/chat/completions


In [11]:
CSV_FILE = os.path.join(config.EXPERIMENTS_DIR, 'experiment_log.csv')

FIELDNAMES = [
    'timestamp',
    'name',
    'goal',
    'model',
    'temperature',
    'max_tokens',
    'top_k',
    'top_p',
    'prompt',
    'output'
]

def init_csv(csv_file: str = CSV_FILE) -> None:
    """Initialize the CSV file with headers if it doesn't exist."""
    if not os.path.exists(csv_file):
        with open(csv_file, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=FIELDNAMES)
            writer.writeheader()

def log_experiment(experiment_data: dict, csv_file: str = CSV_FILE) -> None:
    """Append a new experiment to the CSV log."""
    with open(csv_file, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=FIELDNAMES)
        writer.writerow(experiment_data)

def send_and_log_prompt(
    prompt: str,
    name: str = "None",
    goal: str = "None",
    model: str = "phi-4",
    temperature: float = 0.7,
    max_tokens: int = 500,
    top_k: int = 3,
    top_p: float = 0.1
) -> str:
    """
    Sends a prompt to the local LLM, logs the experiment in a CSV with a timestamp, and returns the response.
    """
    # Prepare model options
    options = {
        'model': model,
        'temperature': temperature,
    }

    headers = config.get_model_config()['headers']()
    payload = {
        'model': model,
        'messages': [{"role": "user", "content": prompt}],
        'temperature': temperature
    }

    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()
        output = response.json()["choices"][0]["message"]["content"].strip()
    except requests.exceptions.RequestException as e:
        print(f"Error connecting to LMStudio: {e}")
        output = "Error"

    # Ensure the CSV exists
    init_csv()

    # Add timestamp to the experiment record
    experiment_record = {
        'timestamp': datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
        'name': name,
        'goal': goal,
        'model': model,
        'temperature': temperature,
        'max_tokens': max_tokens,
        'top_k': top_k,
        'top_p': top_p,
        'prompt': prompt,
        'output': output
    }
    log_experiment(experiment_record)

    return output

In [12]:
full_prompt = """
### Answer the question by SQLite SQL query only and with no explanation. You must minimize SQL execution time while ensuring correctness.
### Sqlite SQL tables, with their properties:
# Products(product_id, parent_product_id, product_name, product_price, product_color, product_size, product_description)
# Customers(customer_id, gender_code, customer_first_name, customer_middle_initial, customer_last_name, email_address, login_name, login_password, phone_number, address_line_1, town_city, county, country)
# Customer_Payment_Methods(customer_id, payment_method_code)
# Invoices(invoice_number, invoice_status_code, invoice_date)
# Orders(order_id, customer_id, order_status_code, date_order_placed)
# Order_Items(order_item_id, product_id, order_id, order_item_status_code)
# Shipments(shipment_id, order_id, invoice_number, shipment_tracking_number, shipment_date)
# Shipment_Items(shipment_id, order_item_id)
#
### Here is some data information about database references.
#
# Products(product_id[1, 2, 3], parent_product_id[8, 3, 1], product_name[Dell monitor, Dell keyboard, iPhone6s], product_price[795.62, 104, 560.93], product_color[Red, Yellow, Red], product_size[Medium, Medium, Small], product_description[Latest model!, Keyboard for games!, Second hand!])
# Customers(customer_id[1, 2, 3], gender_code[Female, Male, Male], customer_first_name[Carmen, Jace, Vickie], customer_middle_initial[K, P, B], customer_last_name[Treutel, Mraz, Bergnaum], email_address[pgulgowski@example.com, zwisozk@example.org, herzog.imogene@example.org], login_name[murphy07, desmond.steuber, kihn.alfonso], login_password[58952d0e0d28de32db3b, 7ba2e47aa0904d9fbdbf, 83a1afbe21f5ca4cd2d5], phone_number[(253)336-6277, 628-468-4228x5917, 633-223-0975], address_line_1[646 Herzog Key Suite 702, 67899 Cassin Hollow Suite 071, 395 Christophe Trail], town_city[Port Madonnahaven, Port Korychester, Lornaland], county[Israel, Palau, Moldova], country[USA, USA, USA])
# Customer_Payment_Methods(customer_id[15, 1, 10], payment_method_code[Direct Debit, Direct Debit, Direct Debit])
# Invoices(invoice_number[1, 2, 3], invoice_status_code[Paid, Issued, Paid], invoice_date[2018-03-09 07:16:07, 2018-01-28 20:08:22, 2018-02-13 02:16:55])
# Orders(order_id[1, 2, 3], customer_id[5, 13, 13], order_status_code[Cancelled, Part Completed, Cancelled], date_order_placed[2017-09-17 16:13:07, 2017-10-14 12:05:48, 2017-09-10 08:27:04])
# Order_Items(order_item_id[1, 2, 3], product_id[4, 3, 2], order_id[8, 4, 7], order_item_status_code[Delivered, Out of Stock, Delivered])
# Shipments(shipment_id[1, 2, 3], order_id[7, 6, 9], invoice_number[5, 2, 4], shipment_tracking_number[6900, 3499, 5617], shipment_date[2018-02-28 00:04:11, 2018-03-07 01:57:14, 2018-03-18 22:23:19])
# Shipment_Items(shipment_id[4, 7, 15], order_item_id[4, 14, 9])
#
### Foreign key information of SQLite tables, used for table joins:
#
# Customer_Payment_Methods(customer_id) REFERENCES Customers(customer_id)
# Orders(customer_id) REFERENCES Customers(customer_id)
# Order_Items(order_id) REFERENCES Orders(order_id)
# Order_Items(product_id) REFERENCES Products(product_id)
# Shipments(order_id) REFERENCES Orders(order_id)
# Shipments(invoice_number) REFERENCES Invoices(invoice_number)
# Shipment_Items(order_item_id) REFERENCES Order_Items(order_item_id)
# Shipment_Items(shipment_id) REFERENCES Shipments(shipment_id)
#
### Some example pairs of questions and corresponding SQL queries are provided based on similar questions:
### How many farms are there?
SELECT count(*) FROM farm
### What is the average, minimum, and maximum age for all French singers?
SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'
### Show the ID of the high schooler named Kyle.
SELECT ID FROM Highschooler WHERE name  =  \"Kyle\"
### What are the names and colors of all products that have been shipped?
"""

In [13]:

response = send_and_log_prompt(
    prompt=full_prompt,
    name="experiment_0",
    goal="Test SQL query generation for NL queries",
    model="phi-4",
    temperature=0.3
)

print("Model response:", response)

Error connecting to LMStudio: 404 Client Error: Not Found for url: http://adrians-mac-mini.local:12345/v1/chat/completions
Model response: Error
